# Fine-tuning - Prédiction de gravité des ACCIDENTS

## Contexte

Ce notebook fait suite au notebook **3-a-accident-initial-ml-training** qui a établi :
- La baseline de performance avec RandomForest et XGBoost
- La comparaison Binary vs Ordered (choix du target binaire)
- L'analyse des feature importances

## Objectif

Optimiser les hyperparamètres pour améliorer les performances au-delà de la baseline.

## Workflow

```
1. Configuration et imports
2. Chargement des données
3. GridSearchCV (recherche des meilleurs hyperparamètres)
4. Impact du nombre d'arbres (n_estimators)
5. Comparaison des algorithmes (RF, XGBoost, LightGBM, CatBoost)
6. Sélection finale et sauvegarde
```

## Critères de sélection

| Critère | Métrique | Justification |
|---------|----------|---------------|
| **Principal** | F1-Score | Équilibre précision/rappel |
| **Secondaire** | AUC | Départage en cas d'égalité |

---

## 1. Configuration et imports

In [1]:
# === IMPORTS ===
import os
import pandas as pd
import numpy as np
import joblib
import warnings

from ml_config import nb_workers, base_estimators, max_evals
from functions import (
    display_metrics, optimize_boosting_model, plot_optimization_history,
    select_best_model, save_best_model
)

# Visualisation
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef,
    classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
)
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# === CONFIGURATION ===
# Seed pour reproductibilité : garantit les mêmes résultats à chaque exécution
RANDOM_STATE = 42

warnings.filterwarnings("ignore", message="pkg_resources is deprecated")
print("Imports OK")

d:\Suivi de formation\simplon-ai-developer-training\W31-ML-DEPLOYMENT\ML\.venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Imports OK


---

## 2. Chargement des données et fonctions utilitaires

In [2]:
# Fonction de chargement (supporte Parquet et CSV)
def load_dataset(name, base_path='data/output'):
    """
    Charge un dataset depuis Parquet (prioritaire) ou CSV (fallback).
    Parquet est préféré car plus rapide et compact.
    """
    parquet_path = f'{base_path}/{name}.parquet'
    csv_path = f'{base_path}/{name}.csv'
    
    if os.path.exists(parquet_path):
        df = pd.read_parquet(parquet_path)
        print(f'{name}: chargé depuis Parquet ({df.shape[0]:,} lignes, {df.shape[1]} colonnes)')
    elif os.path.exists(csv_path):
        df = pd.read_csv(csv_path, sep=';', decimal=',', encoding='utf-8-sig')
        print(f'{name}: chargé depuis CSV ({df.shape[0]:,} lignes, {df.shape[1]} colonnes)')
    else:
        raise FileNotFoundError(f'Dataset {name} non trouvé')
    return df

# Chargement du dataset ACCIDENT
df_accident = load_dataset('dataset_accident')

# Aperçu des données
print(f"\n📊 Colonnes disponibles ({len(df_accident.columns)}):")
print(df_accident.columns.tolist())

dataset_accident: chargé depuis Parquet (263,356 lignes, 13 colonnes)

📊 Colonnes disponibles (13):
['grav_ordered', 'grav_binary', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'region', 'dep', 'agg', 'vma', 'impl_vehicule_leger', 'impl_poids_lourd', 'impl_pieton']


In [3]:
# === FONCTIONS DE PRÉPARATION DES DONNÉES ===

def prepare_data(df, target_col, exclude_cols=None):
    """
    Prépare les features (X) et la target (y) pour l'entraînement.
    
    - Exclut automatiquement toutes les colonnes de gravité (pour éviter le data leakage)
    - Encode les variables catégorielles (region, dep) en entiers
    
    Returns: X (DataFrame), y (Series)
    """
    if exclude_cols is None:
        exclude_cols = []
    
    # Colonnes de gravité à exclure (évite le data leakage)
    target_cols = ['grav', 'grav_ordered', 'grav_binary']
    cols_to_drop = target_cols + exclude_cols
    cols_to_drop = [c for c in cols_to_drop if c in df.columns]
    
    X = df.drop(columns=cols_to_drop)
    y = df[target_col]
    
    # Encodage des variables catégorielles
    for col in X.select_dtypes(include=['object', 'str']).columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    
    return X, y


# === FONCTIONS DE MODÉLISATION ===

def create_pipeline(model):
    """
    Crée un pipeline sklearn : Imputation → Scaling → Modèle.
    Permet d'encapsuler tout le preprocessing avec le modèle.
    """
    return Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('model', model)
    ])


def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Entraîne un modèle et calcule les métriques d'évaluation.
    
    Returns:
        - results: dict avec accuracy, precision, recall, f1, auc
        - y_pred: prédictions sur le test set
        - y_proba: probabilités (pour ROC curve)
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Probabilités pour la courbe ROC
    y_proba = None
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_test)
    
    # Calcul des métriques
    results = {
        'model': model_name,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
        'f1': f1_score(y_test, y_pred, average='weighted', zero_division=0),
    }
    
    # AUC uniquement pour la classification binaire
    if len(np.unique(y_test)) == 2 and y_proba is not None:
        results['auc'] = roc_auc_score(y_test, y_proba[:, 1])
    
    return results, y_pred, y_proba


def evaluate_catboost(catboost_model, X_train, X_test, y_train, y_test, model_name):
    """
    Évalue CatBoost SANS Pipeline sklearn.
    
    Note: CatBoost est incompatible avec sklearn Pipeline depuis sklearn 1.6+
    (problème de sérialisation). On applique donc le preprocessing manuellement.
    """
    # Preprocessing manuel
    imputer = SimpleImputer(strategy='median')
    X_train_imp = imputer.fit_transform(X_train)
    X_test_imp = imputer.transform(X_test)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imp)
    X_test_scaled = scaler.transform(X_test_imp)
    
    # Entraînement et prédiction
    catboost_model.fit(X_train_scaled, y_train)
    y_pred = catboost_model.predict(X_test_scaled)
    
    y_proba = None
    if hasattr(catboost_model, 'predict_proba'):
        y_proba = catboost_model.predict_proba(X_test_scaled)
    
    # Métriques
    results = {
        'model': model_name,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
        'f1': f1_score(y_test, y_pred, average='weighted', zero_division=0),
    }
    
    if len(np.unique(y_test)) == 2 and y_proba is not None:
        results['auc'] = roc_auc_score(y_test, y_proba[:, 1])
    
    return results, y_pred, y_proba


# === FONCTIONS DE VISUALISATION ===

def plot_confusion_matrix(y_true, y_pred, model_name, labels=None):
    """Affiche une matrice de confusion interactive avec Plotly."""
    cm = confusion_matrix(y_true, y_pred)
    
    if labels is None:
        labels = [str(i) for i in sorted(np.unique(y_true))]
    
    # Pourcentages par ligne (= par vraie classe)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Annotations : valeur absolue + pourcentage
    text_annotations = []
    for i in range(len(cm)):
        row = []
        for j in range(len(cm[0])):
            row.append(f"{cm[i][j]}<br>({cm_percent[i][j]:.1f}%)")
        text_annotations.append(row)
    
    fig = go.Figure(data=go.Heatmap(
        z=cm, x=labels, y=labels,
        text=text_annotations, texttemplate="%{text}",
        colorscale='Blues', showscale=True
    ))
    
    fig.update_layout(
        title=f'Matrice de Confusion - {model_name}',
        xaxis_title='Prédiction', yaxis_title='Réalité',
        width=500, height=450
    )
    fig.show()
    return cm


def plot_roc_curve(y_true, y_proba, model_name):
    """Affiche la courbe ROC avec l'AUC (classification binaire uniquement)."""
    if len(np.unique(y_true)) != 2:
        print(f"ROC non disponible pour {model_name}: pas binaire")
        return None
    
    if y_proba is None:
        print(f"ROC non disponible pour {model_name}: pas de probabilités")
        return None
    
    y_score = y_proba[:, 1] if len(y_proba.shape) > 1 else y_proba
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines',
                             name=f'{model_name} (AUC = {roc_auc:.3f})',
                             line=dict(color='blue', width=2)))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
                             name='Random (AUC = 0.5)',
                             line=dict(color='gray', width=1, dash='dash')))
    
    fig.update_layout(
        title=f'Courbe ROC - {model_name} (AUC = {roc_auc:.3f})',
        xaxis_title='Taux de Faux Positifs (FPR)',
        yaxis_title='Taux de Vrais Positifs (TPR)',
        width=550, height=450, showlegend=True
    )
    fig.show()
    return roc_auc


print("✅ Fonctions définies")

✅ Fonctions définies


## 3. Hyperopt - Optimisation bayésienne des hyperparamètres

### Pourquoi Hyperopt plutôt que GridSearchCV ?

| Aspect | GridSearchCV | Hyperopt (TPE) |
|--------|--------------|----------------|
| **Méthode** | Exhaustive (teste tout) | Bayésienne (apprend des essais) |
| **Efficacité** | O(n^k) combinaisons | Converge plus vite |
| **Espace** | Grille discrète | Continu + discret |
| **Intelligence** | Aucune | Mémorise les bons/mauvais essais |

### Algorithme TPE (Tree-structured Parzen Estimator)

1. Évalue quelques points aléatoires
2. Sépare les résultats en "bons" et "mauvais"
3. Modélise P(params | bons) et P(params | mauvais)
4. Choisit le prochain point qui maximise le ratio

In [4]:
# === HYPEROPT - OPTIMISATION BAYÉSIENNE ===

# Préparation des données
X_acc_bin, y_acc_bin = prepare_data(df_accident, 'grav_binary')
X_train_acc_bin, X_test_acc_bin, y_train_acc_bin, y_test_acc_bin = train_test_split(
    X_acc_bin, y_acc_bin, test_size=0.2, random_state=RANDOM_STATE, stratify=y_acc_bin
)

# Sous-échantillonnage pour accélérer (50k)
sample_size_acc = min(50000, len(X_train_acc_bin))
X_sample_acc = X_train_acc_bin.sample(n=sample_size_acc, random_state=RANDOM_STATE)
y_sample_acc = y_train_acc_bin.loc[X_sample_acc.index]

print(f"Hyperopt sur {sample_size_acc:,} echantillons")
print("=" * 50)

# === RANDOMFOREST ===
best_params_rf, best_f1_rf, trials_rf = optimize_boosting_model(
    X_sample_acc, y_sample_acc,
    model_type='randomforest',
    max_evals=max_evals,
    cv=3,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=nb_workers
)

# Validation sur test set complet
print("\nValidation sur test set complet...")
best_model_rf = create_pipeline(RandomForestClassifier(
    **best_params_rf,
    random_state=RANDOM_STATE,
    n_jobs=nb_workers,
    class_weight='balanced'
))
best_model_rf.fit(X_train_acc_bin, y_train_acc_bin)
y_pred_rf = best_model_rf.predict(X_test_acc_bin)
f1_rf_test = f1_score(y_test_acc_bin, y_pred_rf)
print(f"F1 sur test set: {f1_rf_test:.3f}")


Hyperopt sur 50,000 echantillons
Optimisation Hyperopt pour RANDOMFOREST...
  - max_evals: 50
  - cv: 3 folds
  - scoring: f1

100%|██████████| 50/50 [47:36<00:00, 57.12s/trial, best loss: -0.642990309783856] 

Meilleurs parametres RANDOMFOREST:
  - max_depth: 1
  - max_features: 2
  - min_samples_leaf: 10
  - min_samples_split: 11
  - n_estimators: 350

Meilleur f1 (CV): 0.6430 (+/- 0.0009)

Validation sur test set complet...
F1 sur test set: 0.527


### 3.2 Hyperopt - CatBoost

CatBoost utilise des hyperparametres differents de RandomForest :
- **iterations** (n_estimators) : nombre d'arbres (attention a l'overfitting)
- **learning_rate** : pas d'apprentissage (plus petit = plus lent mais plus precis)
- **depth** : profondeur max (typiquement 4-10 pour boosting)
- **l2_leaf_reg** : regularisation L2

In [5]:
# === HYPEROPT - CATBOOST ===

best_params_cb, best_f1_cb, trials_cb = optimize_boosting_model(
    X_sample_acc, y_sample_acc,
    model_type='catboost',
    max_evals=max_evals,
    cv=3,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=nb_workers
)

# Validation sur test set complet
print("\nValidation sur test set complet...")
imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train_acc_bin),
    columns=X_train_acc_bin.columns
)
X_test_imp = pd.DataFrame(
    imputer.transform(X_test_acc_bin),
    columns=X_test_acc_bin.columns
)

best_model_cb = CatBoostClassifier(
    **best_params_cb,
    random_state=RANDOM_STATE,
    auto_class_weights='Balanced',
    verbose=False
)
best_model_cb.fit(X_train_imp, y_train_acc_bin)
y_pred_cb = best_model_cb.predict(X_test_imp)
f1_cb_test = f1_score(y_test_acc_bin, y_pred_cb)
print(f"F1 sur test set: {f1_cb_test:.3f}")


Optimisation Hyperopt pour CATBOOST...
  - max_evals: 50
  - cv: 3 folds
  - scoring: f1

100%|██████████| 50/50 [00:00<00:00, 301.95trial/s, best loss: 0.0]

Meilleurs parametres CATBOOST:
  - depth: 8
  - iterations: 450
  - l2_leaf_reg: 4.169836222804728
  - learning_rate: 0.15917237508778656

Meilleur f1 (CV): 0.0000 (+/- 0.0000)

Validation sur test set complet...
F1 sur test set: 0.655


### 3.3 Hyperopt - XGBoost

XGBoost (eXtreme Gradient Boosting) utilise des hyperparamètres similaires à CatBoost :
- **n_estimators** : nombre d'arbres
- **learning_rate** : taux d'apprentissage
- **max_depth** : profondeur max des arbres
- **min_child_weight** : poids minimum par feuille (régularisation)
- **subsample** / **colsample_bytree** : échantillonnage pour réduire l'overfitting

In [6]:
best_params_xgb, best_f1_xgb, trials_xgb = optimize_boosting_model(
    X_sample_acc, y_sample_acc,
    model_type='xgboost',
    max_evals=max_evals,
    cv=3,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=nb_workers
)

# Validation sur test set complet
print("\nValidation sur test set complet...")
imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train_acc_bin),
    columns=X_train_acc_bin.columns
)
X_test_imp = pd.DataFrame(
    imputer.transform(X_test_acc_bin),
    columns=X_test_acc_bin.columns
)

best_model_xgb = XGBClassifier(
    **best_params_xgb,
    random_state=RANDOM_STATE,
    scale_pos_weight=18 / 10,
    verbosity=0,
)
best_model_xgb.fit(X_train_imp, y_train_acc_bin)
y_pred_xgb = best_model_xgb.predict(X_test_imp)
f1_xgb_test = f1_score(y_test_acc_bin, y_pred_xgb)
print(f"F1 sur test set: {f1_xgb_test:.3f}")

Optimisation Hyperopt pour XGBOOST...
  - max_evals: 50
  - cv: 3 folds
  - scoring: f1

100%|██████████| 50/50 [02:41<00:00,  3.22s/trial, best loss: -0.6513562417010096]

Meilleurs parametres XGBOOST:
  - colsample_bytree: 0.6011993082667989
  - learning_rate: 0.09687815138534953
  - max_depth: 6
  - min_child_weight: 8
  - n_estimators: 250
  - subsample: 0.9421080655965379

Meilleur f1 (CV): 0.6514 (+/- 0.0026)

Validation sur test set complet...
F1 sur test set: 0.655


### 3.4 Hyperopt - LightGBM

LightGBM (Light Gradient Boosting Machine) est optimisé pour la vitesse :
- **n_estimators** : nombre d'arbres
- **learning_rate** : taux d'apprentissage
- **max_depth** : profondeur max (peut être -1 pour illimité)
- **num_leaves** : nombre max de feuilles par arbre (spécifique à LightGBM)
- **min_child_samples** : échantillons minimum par feuille

In [7]:
best_params_lgbm, best_f1_lgbm, trials_lgbm = optimize_boosting_model(
    X_sample_acc, y_sample_acc,
    model_type='lightgbm',
    max_evals=max_evals,
    cv=3,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=nb_workers
)

# Validation sur test set complet
print("\nValidation sur test set complet...")
imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(
    imputer.fit_transform(X_train_acc_bin),
    columns=X_train_acc_bin.columns
)
X_test_imp = pd.DataFrame(
    imputer.transform(X_test_acc_bin),
    columns=X_test_acc_bin.columns
)

best_model_lgbm = LGBMClassifier(
    **best_params_lgbm,
    random_state=RANDOM_STATE,
    class_weights='balanced',
    verbose=-1
)
best_model_lgbm.fit(X_train_imp, y_train_acc_bin)
y_pred_lgbm = best_model_lgbm.predict(X_test_imp)
f1_lgbm_test = f1_score(y_test_acc_bin, y_pred_xgb)
print(f"F1 sur test set: {f1_lgbm_test:.3f}")

Optimisation Hyperopt pour LIGHTGBM...
  - max_evals: 50
  - cv: 3 folds
  - scoring: f1

100%|██████████| 50/50 [02:03<00:00,  2.47s/trial, best loss: -0.6014562598198895]

Meilleurs parametres LIGHTGBM:
  - colsample_bytree: 0.7458197154615357
  - learning_rate: 0.1095092638225987
  - max_depth: 8
  - min_child_samples: 40
  - n_estimators: 250
  - num_leaves: 50
  - subsample: 0.8576876180176862

Meilleur f1 (CV): 0.6015 (+/- 0.0004)

Validation sur test set complet...
F1 sur test set: 0.655


### 3.5 Convergence de l'optimisation

Visualisation de la progression d'Hyperopt pour chaque algorithme.

**Interprétation :**
- Les points représentent chaque essai (combinaison d'hyperparamètres)
- La courbe rouge montre le meilleur score cumulé
- Une courbe qui se stabilise indique que l'espace a été bien exploré

In [8]:
# === VISUALISATION DE LA CONVERGENCE HYPEROPT ===

fig = make_subplots(rows=1, cols=2, subplot_titles=['RandomForest', 'CatBoost'])

# RF
scores_rf = [t['result']['score'] for t in trials_rf.trials]
best_rf = np.maximum.accumulate(scores_rf)
fig.add_trace(go.Scatter(y=scores_rf, mode='markers', name='RF essais', marker=dict(size=5, opacity=0.5)), row=1, col=1)
fig.add_trace(go.Scatter(y=best_rf, mode='lines', name='RF best', line=dict(color='red', width=2)), row=1, col=1)

# CatBoost
scores_cb = [t['result']['score'] for t in trials_cb.trials]
best_cb = np.maximum.accumulate(scores_cb)
fig.add_trace(go.Scatter(y=scores_cb, mode='markers', name='CB essais', marker=dict(size=5, opacity=0.5)), row=1, col=2)
fig.add_trace(go.Scatter(y=best_cb, mode='lines', name='CB best', line=dict(color='red', width=2)), row=1, col=2)

# XGBoost
scores_xgb = [t['result']['score'] for t in trials_xgb.trials]
best_xgb = np.maximum.accumulate(scores_xgb)
fig.add_trace(go.Scatter(y=scores_xgb, mode='markers', name='xgb essais', marker=dict(size=5, opacity=0.5)), row=1, col=2)
fig.add_trace(go.Scatter(y=best_xgb, mode='lines', name='xgb best', line=dict(color='red', width=2)), row=1, col=2)

# LightGBM
scores_lgbm = [t['result']['score'] for t in trials_lgbm.trials]
best_lgbm = np.maximum.accumulate(scores_lgbm)
fig.add_trace(go.Scatter(y=scores_lgbm, mode='markers', name='lgbm essais', marker=dict(size=5, opacity=0.5)), row=1, col=2)
fig.add_trace(go.Scatter(y=best_lgbm, mode='lines', name='lgbm best', line=dict(color='red', width=2)), row=1, col=2)

fig.update_layout(title='Convergence Hyperopt', height=400, width=900, showlegend=False)
fig.update_xaxes(title_text='Essai')
fig.update_yaxes(title_text='F1 Score')
fig.show()

# Résumé
print("\n" + "=" * 50)
print("RESUME HYPEROPT")
print("=" * 50)
print(f"RandomForest : F1 CV={best_f1_rf:.3f} | F1 test={f1_rf_test:.3f}")
print(f"CatBoost     : F1 CV={best_f1_cb:.3f} | F1 test={f1_cb_test:.3f}")
print(f"XGBoost      : F1 CV={best_f1_xgb:.3f} | F1 test={f1_xgb_test:.3f}")
print(f"LightGBM     : F1 CV={best_f1_lgbm:.3f} | F1 test={f1_lgbm_test:.3f}")



RESUME HYPEROPT
RandomForest : F1 CV=0.643 | F1 test=0.527
CatBoost     : F1 CV=0.000 | F1 test=0.655
XGBoost      : F1 CV=0.651 | F1 test=0.655
LightGBM     : F1 CV=0.601 | F1 test=0.655


## 4. Impact du nombre d'arbres (`n_estimators`)

**Question :** Au-delà de 300 arbres, y a-t-il encore un gain de performance ?

**Attendu :** Les performances devraient se stabiliser au-delà d'un certain seuil (loi des rendements décroissants).

In [9]:
# Test nombre d'arbres sur ACCIDENT - TOUTES LES MÉTRIQUES
n_estimators_list_acc = [20, 50, 100, 200]
results_trees_acc = []

for n_trees in n_estimators_list_acc:
    print(f"Testing n_estimators={n_trees} sur ACCIDENT...", end=" ")
    
    model_acc = create_pipeline(RandomForestClassifier(
        n_estimators=n_trees, random_state=RANDOM_STATE, n_jobs=nb_workers, class_weight='balanced'
    ))
    
    res, y_pred_tree, y_proba_tree = evaluate_model(model_acc, X_train_acc_bin, X_test_acc_bin, 
                               y_train_acc_bin, y_test_acc_bin, f"RF_{n_trees}")
    res['n_estimators'] = n_trees
    
    # Métriques supplémentaires
    cm = confusion_matrix(y_test_acc_bin, y_pred_tree)
    tn, fp, fn, tp = cm.ravel()
    res['specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0
    res['mcc'] = matthews_corrcoef(y_test_acc_bin, y_pred_tree)
    
    results_trees_acc.append(res)
    print(f"F1={res['f1']:.3f} | Recall={res['recall']:.3f} | AUC={res.get('auc', 0):.3f}")

df_trees_acc = pd.DataFrame(results_trees_acc)
print("\n📊 Tableau complet ACCIDENT:")
print(df_trees_acc[['n_estimators', 'accuracy', 'precision', 'recall', 'specificity', 'f1', 'auc', 'mcc']].round(3))

# Graphique avec toutes les métriques
metrics_to_plot = ['accuracy', 'precision', 'recall', 'specificity', 'f1', 'auc', 'mcc']
fig = go.Figure()

for metric in metrics_to_plot:
    if metric in df_trees_acc.columns:
        fig.add_trace(go.Scatter(
            x=df_trees_acc['n_estimators'],
            y=df_trees_acc[metric],
            mode='lines+markers',
            name=metric.upper(),
            text=[f"{v:.3f}" for v in df_trees_acc[metric]],
            hovertemplate=f'{metric}: %{{y:.3f}}<extra></extra>'
        ))

fig.update_layout(
    title='Impact du nombre d\'arbres - ACCIDENT (toutes métriques)',
    xaxis_title='Nombre d\'arbres (n_estimators)',
    yaxis_title='Score',
    yaxis_range=[0, 1],
    width=900,
    height=500,
    legend_title='Métrique'
)
fig.show()

Testing n_estimators=20 sur ACCIDENT... F1=0.715 | Recall=0.712 | AUC=0.753
Testing n_estimators=50 sur ACCIDENT... F1=0.715 | Recall=0.713 | AUC=0.754
Testing n_estimators=100 sur ACCIDENT... F1=0.716 | Recall=0.713 | AUC=0.755
Testing n_estimators=200 sur ACCIDENT... F1=0.716 | Recall=0.713 | AUC=0.755

📊 Tableau complet ACCIDENT:
   n_estimators  accuracy  precision  recall  specificity     f1    auc    mcc
0            20     0.712      0.720   0.712        0.752  0.715  0.753  0.384
1            50     0.713      0.720   0.713        0.751  0.715  0.754  0.385
2           100     0.713      0.722   0.713        0.750  0.716  0.755  0.388
3           200     0.713      0.721   0.713        0.748  0.716  0.755  0.387


## 5. Comparaison des algorithmes de boosting

**Algorithmes testés :**

| Algorithme | Avantages | Inconvénients |
|------------|-----------|---------------|
| **RandomForest** | Robuste, interprétable, peu d'hyperparamètres | Peut être moins performant |
| **XGBoost** | Très performant, régularisation intégrée | Plus lent à entraîner |
| **LightGBM** | Très rapide, gère bien les grandes données | Peut overfitter |
| **CatBoost** | Gère nativement les catégorielles | Incompatible sklearn Pipeline (1.6+) |

**Objectif :** Identifier l'algorithme avec le meilleur compromis F1/AUC.

In [10]:
# === COMPARAISON DES 6 MODÈLES ACCIDENT ===
print("="*70)
print("COMPARAISON DES MODÈLES ACCIDENT (6 modèles)")
print("="*70)

models_eval_acc = []

# 1. RF_baseline
rf_baseline = create_pipeline(RandomForestClassifier(
    n_estimators=base_estimators,
    random_state=RANDOM_STATE,
    n_jobs=nb_workers,
    class_weight='balanced'
))
res, y_pred, y_proba = evaluate_model(rf_baseline, X_train_acc_bin, X_test_acc_bin,
                                      y_train_acc_bin, y_test_acc_bin, 'RF_baseline')
models_eval_acc.append({"name": "RF_baseline", "y_true": y_test_acc_bin, "y_pred": y_pred, "y_proba": y_proba})

# 2. XGB_baseline
xgb_baseline = create_pipeline(XGBClassifier(
    n_estimators=base_estimators,
    random_state=RANDOM_STATE,
    n_jobs=nb_workers,
    scale_pos_weight=len(y_train_acc_bin[y_train_acc_bin==0])/len(y_train_acc_bin[y_train_acc_bin==1])
))
res, y_pred, y_proba = evaluate_model(xgb_baseline, X_train_acc_bin, X_test_acc_bin,
                                      y_train_acc_bin, y_test_acc_bin, 'XGB_baseline')
models_eval_acc.append({"name": "XGB_baseline", "y_true": y_test_acc_bin, "y_pred": y_pred, "y_proba": y_proba})

# 3. RF_HyperOpt (RandomForest optimisé)
rf_hyperopt = create_pipeline(RandomForestClassifier(
    n_estimators=best_params_rf.get('n_estimators', 200),
    max_depth=best_params_rf.get('max_depth', None),
    min_samples_split=best_params_rf.get('min_samples_split', 2),
    random_state=RANDOM_STATE,
    n_jobs=nb_workers,
    class_weight='balanced'
))
res, y_pred, y_proba = evaluate_model(rf_hyperopt, X_train_acc_bin, X_test_acc_bin,
                                      y_train_acc_bin, y_test_acc_bin, 'RF_HyperOpt')
models_eval_acc.append({"name": "RF_HyperOpt", "y_true": y_test_acc_bin, "y_pred": y_pred, "y_proba": y_proba})

# 4. XGBoost optimisé
xgb_opt = create_pipeline(XGBClassifier(
    **best_params_xgb,
    random_state=RANDOM_STATE,
    n_jobs=nb_workers,
    scale_pos_weight=len(y_train_acc_bin[y_train_acc_bin==0])/len(y_train_acc_bin[y_train_acc_bin==1])
))
res, y_pred, y_proba = evaluate_model(xgb_opt, X_train_acc_bin, X_test_acc_bin,
                                      y_train_acc_bin, y_test_acc_bin, 'XGBoost')
models_eval_acc.append({"name": "XGBoost", "y_true": y_test_acc_bin, "y_pred": y_pred, "y_proba": y_proba})

# 5. LightGBM optimisé
lgbm_opt = create_pipeline(LGBMClassifier(
    **best_params_lgbm,
    random_state=RANDOM_STATE,
    n_jobs=nb_workers,
    class_weight='balanced',
    verbose=-1
))
res, y_pred, y_proba = evaluate_model(lgbm_opt, X_train_acc_bin, X_test_acc_bin,
                                      y_train_acc_bin, y_test_acc_bin, 'LightGBM')
models_eval_acc.append({"name": "LightGBM", "y_true": y_test_acc_bin, "y_pred": y_pred, "y_proba": y_proba})
# 6. CatBoost optimisé (sans Pipeline - incompatible sklearn 1.6+)
catboost_opt = CatBoostClassifier(
    **best_params_cb,
    random_state=RANDOM_STATE,
    auto_class_weights='Balanced',
    verbose=False
)
res, y_pred, y_proba = evaluate_catboost(catboost_opt, X_train_acc_bin, X_test_acc_bin,
                                         y_train_acc_bin, y_test_acc_bin, 'CatBoost')
models_eval_acc.append({"name": "CatBoost", "y_true": y_test_acc_bin, "y_pred": y_pred, "y_proba": y_proba})

# Affichage complet avec display_metrics (confusion matrices, ROC, bar chart, table)
display_metrics(
    models_results=models_eval_acc,
    class_labels=["Non grave", "Grave"]
)

COMPARAISON DES MODÈLES ACCIDENT (6 modèles)


d:\Suivi de formation\simplon-ai-developer-training\W31-ML-DEPLOYMENT\ML\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names

d:\Suivi de formation\simplon-ai-developer-training\W31-ML-DEPLOYMENT\ML\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names



EVALUATION DES MODELES (6 modele(s))

📊 TABLEAU DES METRIQUES
----------------------------------------------------------------------
       model  accuracy  balanced_accuracy  precision  recall     f1  roc_auc    mcc  specificity  sensitivity
 RF_baseline    0.7134             0.6981     0.7216  0.7134 0.7164   0.7548 0.3878       0.7496       0.6465
XGB_baseline    0.7254             0.7281     0.7477  0.7254 0.7307   0.8019 0.4386       0.7188       0.7374
 RF_HyperOpt    0.6639             0.6305     0.6634  0.6639 0.6637   0.6907 0.2614       0.7430       0.5180
     XGBoost    0.7257             0.7295     0.7491  0.7257 0.7312   0.8019 0.4410       0.7167       0.7424
    LightGBM    0.7257             0.7293     0.7488  0.7257 0.7311   0.8022 0.4406       0.7172       0.7414
    CatBoost    0.7263             0.7290     0.7485  0.7263 0.7316   0.8012 0.4403       0.7199       0.7381




📈 MATRICES DE CONFUSION
----------------------------------------------------------------------



📉 COURBES ROC
----------------------------------------------------------------------



RESUME
Meilleur modele (F1): CatBoost (F1 = 0.7316)
Meilleur modele (AUC): LightGBM (AUC = 0.8022)


{'metrics_df':           model  accuracy  balanced_accuracy  precision  recall      f1  \
 0   RF_baseline    0.7134             0.6981     0.7216  0.7134  0.7164   
 1  XGB_baseline    0.7254             0.7281     0.7477  0.7254  0.7307   
 2   RF_HyperOpt    0.6639             0.6305     0.6634  0.6639  0.6637   
 3       XGBoost    0.7257             0.7295     0.7491  0.7257  0.7312   
 4      LightGBM    0.7257             0.7293     0.7488  0.7257  0.7311   
 5      CatBoost    0.7263             0.7290     0.7485  0.7263  0.7316   
 
    roc_auc     mcc  specificity  sensitivity  
 0   0.7548  0.3878       0.7496       0.6465  
 1   0.8019  0.4386       0.7188       0.7374  
 2   0.6907  0.2614       0.7430       0.5180  
 3   0.8019  0.4410       0.7167       0.7424  
 4   0.8022  0.4406       0.7172       0.7414  
 5   0.8012  0.4403       0.7199       0.7381  ,
 'figures': {'bar_chart': Figure({
      'data': [{'marker': {'color': '#3498db'},
                'name': 'RF_base

## 6. Comparaison finale et sélection du meilleur modèle

**Critères de sélection :**
1. **F1-Score** (critère principal)
2. **AUC** (départage en cas d'égalité sur F1)

**Modèles comparés :**
- RandomForest baseline
- RandomForest optimisé (meilleurs hyperparamètres)
- RandomForest avec plus d'arbres
- XGBoost
- LightGBM
- CatBoost

In [11]:
# Sélection du meilleur modèle ACCIDENT
print("="*70)
print("SÉLECTION DU MEILLEUR MODÈLE ACCIDENT")
print("="*70)

best_model_info = select_best_model(models_eval_acc, metric='f1')
print(f"\nMeilleur modèle: {best_model_info['name']}")
print(f"F1-Score: {best_model_info['score']:.4f}")

SÉLECTION DU MEILLEUR MODÈLE ACCIDENT

Meilleur modèle: CatBoost
F1-Score: 0.7316


### 6.1 Interprétation des matrices de confusion

**Lecture de la matrice :**
- **TP (True Positive)** : Accidents graves correctement détectés → Bon !
- **FN (False Negative)** : Accidents graves manqués → Dangereux (sous-estimation du risque)
- **FP (False Positive)** : Fausses alertes → Coûteux mais pas dangereux
- **TN (True Negative)** : Non-graves correctement identifiés → Bon !

**Compromis à faire :**
- Privilégier le **Recall** si on veut minimiser les FN (ne pas rater de cas graves)
- Privilégier la **Precision** si on veut minimiser les FP (éviter les fausses alertes)

## 7. Sauvegarde modeles optimises

In [12]:
# Sauvegarder le meilleur modèle ACCIDENT
print("="*70)
print("SAUVEGARDE DU MEILLEUR MODÈLE ACCIDENT")
print("="*70)

# Configuration des modèles (même config que la comparaison)
X_acc_bin_full, y_acc_bin_full = prepare_data(df_accident, 'grav_binary')
pos_weight = len(y_acc_bin_full[y_acc_bin_full==0]) / len(y_acc_bin_full[y_acc_bin_full==1])

model_configs = {
    'RF_baseline': lambda: create_pipeline(RandomForestClassifier(
        n_estimators=base_estimators, random_state=RANDOM_STATE,
        n_jobs=nb_workers, class_weight='balanced'
    )),
    'XGB_baseline': lambda: create_pipeline(XGBClassifier(
        n_estimators=base_estimators, random_state=RANDOM_STATE,
        n_jobs=nb_workers, scale_pos_weight=pos_weight
    )),
    'RF_HyperOpt': lambda: create_pipeline(RandomForestClassifier(
        n_estimators=best_params_rf.get('n_estimators', 200),
        max_depth=best_params_rf.get('max_depth', None),
        min_samples_split=best_params_rf.get('min_samples_split', 2),
        random_state=RANDOM_STATE, n_jobs=nb_workers, class_weight='balanced'
    )),
    'XGBoost': lambda: create_pipeline(XGBClassifier(
        **best_params_xgb, random_state=RANDOM_STATE,
        n_jobs=nb_workers, scale_pos_weight=pos_weight
    )),
    'LightGBM': lambda: create_pipeline(LGBMClassifier(
        **best_params_lgbm, random_state=RANDOM_STATE,
        n_jobs=nb_workers, class_weight='balanced', verbose=-1
    )),
    'CatBoost': lambda: CatBoostClassifier(
        **best_params_cb, random_state=RANDOM_STATE,
        auto_class_weights='Balanced', verbose=False
    )
}

result = save_best_model(
    best_model_name=best_model_info['name'],
    model_configs=model_configs,
    X_full=X_acc_bin_full,
    y_full=y_acc_bin_full,
    X_test=X_test_acc_bin,
    y_test=y_test_acc_bin,
    save_path='models/model_accident_binary_optimized.joblib'
)

SAUVEGARDE DU MEILLEUR MODÈLE ACCIDENT
Entrainement du modele: CatBoost
Sauvegarde: models/model_accident_binary_optimized.joblib
F1 sur test (weighted): 0.7421

Features attendues en entree (11):
  1. est_nuit
  2. est_heure_pointe
  3. jour_semaine
  4. est_weekend
  5. region
  6. dep
  7. agg
  8. vma
  9. impl_vehicule_leger
  10. impl_poids_lourd
  11. impl_pieton


## 8. Résumé et conclusions

### Modèles sauvegardés

| Fichier | Description | Usage |
|---------|-------------|-------|
|  | Classification binaire - baseline | Fallback |
|  | Binaire optimisé (Hyperopt) | **Production** |

### Résultats de l'optimisation

| Algorithme | F1 CV (Hyperopt) | F1 Test | Commentaire |
|------------|------------------|---------|-------------|
| RandomForest | ~0.64 | ~0.53 | Overfitting possible |
| XGBoost | ~0.65 | ~0.66 | Bon compromis |
| LightGBM | ~0.60 | ~0.66 | Rapide et performant |
| CatBoost | - | ~0.66 | **Sélectionné** |

### Recommandations pour l'API

1. **Endpoint principal** : Utiliser 
2. **Preprocessing** : Appliquer le même pipeline (imputation + scaling) qu'à l'entraînement
3. **Seuil de décision** : Par défaut 0.5, ajustable selon le compromis FP/FN souhaité

### Pistes d'amélioration future

- Enrichir les features avec des données externes (météo détaillée, trafic)
- Tester d'autres architectures (stacking, neural networks)
- Optimiser le seuil de décision selon les coûts métier (FN plus grave que FP)